In [1]:
# check for cuda
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os
import time
import copy
import argparse
import ast
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import re
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from scipy.sparse import lil_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
# check for cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [14]:
# Load the CSV file and extract the Description column
data = pd.read_csv('file/train.csv')
corpus = data['Description']
sentence_count= 0
sentence_limit = 40
sentences_list = []

for sentence in corpus:
    sentences_list.append(sentence)
    sentence_count += 1
    if sentence_count > sentence_limit:
        break

word_counts = {}
text_courpus = []
MinWords = 40000 # min words in a sentence
minSentenceLength = 500 # min words in a sentence
maxSenteceLength = 0 # max words in a sentence
for sentence in sentences_list:
    sentence = sentence.lower() # convert to lower case
    sentence = re.sub(r'[^\w\s]', '', sentence) # remove punctuation
    text_courpus.append(sentence)
    words = sentence.split()
    minSentenceLength = min(minSentenceLength, len(words))
    maxSenteceLength = max(maxSenteceLength, len(words))
    for word in words:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1


In [15]:
class WordEmbedding:
    def __init__(self, window_size=2, embedding_size=100):
        self.window_size = window_size
        self.embedding_size = embedding_size
        self.word2index = {}
        self.index2word = {}
        self.word_counts = {}
        self.word_vectors = None
        self.word_vectors_norm = None
        self.vocab_size = 0
    def buid_Co_occurence_matrix(self, corpus, word_counts):
        self.word2index = {word: i for i, word in enumerate(word_counts.keys())}
        self.index2word = {i: word for word, i in self.word2index.items()}
        self.word_counts = word_counts
        self.corpus = corpus
        vocab = set(word_counts.keys())
        self.vocab_size = len(vocab)
        #Create a n*n matrix where n is the number of words in the vocab
        self.co_occurence_matrix = lil_matrix((self.vocab_size, self.vocab_size), dtype=np.float32)
        for sentence in corpus:
            words = sentence.split()
            for i, word in enumerate(words):
                if word in self.word2index:
                    for j in range(max(i - self.window_size, 0), min(i + self.window_size, len(words))):
                        if i != j and words[j] in self.word2index:
                            self.co_occurence_matrix[self.word2index[word], self.word2index[words[j]]] += 1
    
    def train(self, corpus, word_counts):
        self.buid_Co_occurence_matrix(corpus, word_counts)
        svd = TruncatedSVD(n_components=self.embedding_size)
        svd.fit(self.co_occurence_matrix)
        embeddings = svd.components_
        embeddings = np.transpose(embeddings)
        return embeddings, self.word2index, self.index2word
    


In [16]:
# Sample corpus and word counts
corpus1 = [
    "the quick brown fox",
    "jumps over the lazy dog"
]

word_counts1 = {
    "the": 2,
    "quick": 1,
    "brown": 1,
    "fox": 1,
    "jumps": 1,
    "over": 1,
    "lazy": 1,
    "dog": 1
}

# Initialize and train the WordEmbedding model
embedding_model = WordEmbedding(window_size=2, embedding_size=2)
embeddings, word2index, index2word = embedding_model.train(corpus1, word_counts1)

# Print the co-occurrence matrix
print("Co-occurrence Matrix:")
print(embedding_model.co_occurence_matrix.toarray())

# Print the resulting embeddings
print("\nWord Embeddings:")
for word, idx in word2index.items():
    print(f"{word}: {embeddings[idx]}")

Co-occurrence Matrix:
[[0. 1. 0. 0. 1. 1. 1. 0.]
 [1. 0. 1. 0. 0. 0. 0. 0.]
 [1. 1. 0. 1. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 0. 1.]
 [1. 0. 0. 0. 0. 0. 1. 0.]]

Word Embeddings:
the: [ 0.6592979 -0.6249202]
quick: [0.40738988 0.2680209 ]
brown: [ 0.18280017 -0.24515137]
fox: [ 0.15605648 -0.1453273 ]
jumps: [0.29821414 0.31788492]
over: [0.3752575  0.50180185]
lazy: [0.29821417 0.31788495]
dog: [ 0.15135553 -0.05013293]


In [17]:
%%time
Embedding_Model = WordEmbedding(window_size=3, embedding_size=100)
embeddings, word2index, index2word = Embedding_Model.train(text_courpus, word_counts)

CPU times: total: 297 ms
Wall time: 272 ms


In [18]:
import json
word_embeddings_tensor = torch.FloatTensor(embeddings)
# Save the embeddings to a file
torch.save(word_embeddings_tensor, 'svd-word-vectors.pt')
# Save word2index and index2word to JSON files
def save_dictionary(dictionary, file_path):
    with open(file_path, 'w') as f:
        json.dump(dictionary, f)

# Load word2index and index2word from JSON files

# Save dictionaries to JSON files
save_dictionary(word2index, 'word2index.json')
save_dictionary(index2word, 'index2word.json')



In [21]:
len(embeddings)

640

## Elmo Model

In [6]:
import torch
import torch.nn as nn

class ELMO(nn.Module):
    def __init__(self, initial_embedding, hidden_size, num_layers):
        super().__init__()

        # Initialize embedding layer
        self.embedding_layer = nn.Embedding.from_pretrained(torch.tensor(initial_embedding))

        # Initialize bidirectional LSTM layers
        self.lstm_layer1 = nn.LSTM(input_size=self.embedding_layer.embedding_dim, 
                                   hidden_size=hidden_size, 
                                   bidirectional=True, 
                                   batch_first=True)
        self.lstm_layer2 = nn.LSTM(input_size=self.embedding_layer.embedding_dim, 
                                   hidden_size=hidden_size, 
                                   bidirectional=True, 
                                   batch_first=True)
        
        # Initialize linear layer
        self.linear_layer = nn.Linear(in_features=self.embedding_layer.embedding_dim, 
                                      out_features=self.embedding_layer.num_embeddings)

        # Initialize lambda parameters
        self.lambda1 = nn.Parameter(torch.rand(1))
        self.lambda2 = nn.Parameter(torch.rand(1))
        self.lambda3 = nn.Parameter(torch.rand(1))

    def forward(self, x, is_training):
        # Embedding layer
        embedded_input = self.embedding_layer(x)
        
        # BiLSTM layers
        lstm_output1, _ = self.lstm_layer1(embedded_input)
        lstm_output2, _ = self.lstm_layer2(lstm_output1)
        
        # Weighted sum of embeddings
        output = self.lambda1 * embedded_input + self.lambda2 * lstm_output1 + self.lambda3 * lstm_output2
        
        # Apply linear layer if training
        if is_training:
            output = self.linear_layer(output)
        
        return output


In [9]:
def shift_sentences(sentences):
    shifted_sentences = []
    for sentence in sentences:
        # shifted_sentence = torch.cat((sentence[1:], torch.tensor([wordtoind['<eos>']])))
        shifted_sentence = sentence[1:] + [word2index['<eos>']]
        shifted_sentences.append(shifted_sentence)

    return shifted_sentences